## Spark DataFrame Basics

In [1]:
import findspark
import os

In [2]:
findspark.init('/home/siddharth/spark-2.4.1-bin-hadoop2.7/')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('basics').getOrCreate()

In [5]:
dataset_path = './Python-and-Spark-for-Big-Data-master/Spark_DataFrames/'
os.chdir(dataset_path)


In [6]:
os.getcwd()

'/home/siddharth/workspace-python/PySpark_ml/Python-and-Spark-for-Big-Data-master/Spark_DataFrames'

##### Be carefull while giving spark read path as os.chdir() doesn't work [jupyter and file should be at same location to work]

In [7]:
df = spark.read.json('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json')

In [8]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [9]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [11]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [12]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [13]:
df.columns

['age', 'name']

##### for multiple import in new line use "()" paranthesis

In [14]:
from pyspark.sql.types import (StructField,StructType,
                               StringType,IntegerType)

##### True is for null allowance

In [15]:
data_schema = [StructField('age',IntegerType(),True),
               StructField('name',StringType(),True)]

In [16]:
final_struc = StructType(fields=data_schema)

In [17]:
df_n = spark.read.json('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json',schema=final_struc)

In [18]:
df_n.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



## Basics Part 2

In [19]:
# column object

type(df_n['age'])

pyspark.sql.column.Column

In [22]:
df_n['age']

Column<b'age'>

In [20]:
df_n.select('age')

DataFrame[age: bigint]

In [21]:
df_n.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [22]:
type(df_n.select('age'))

pyspark.sql.dataframe.DataFrame

##### column objects are not callable hence don't use show()

In [23]:
#df_n['age'].show()

In [24]:
df_n.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [25]:
df_n.head(2)[0]

Row(age=None, name='Michael')

In [26]:
type(df_n.head(2)[0])

pyspark.sql.types.Row

In [27]:
df_n.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



##### returns a 'new' dataframe either by adding a new column or replacing an old column [without show()]
##### NOT an inplace operation

In [28]:
df_n.withColumn('double_age',df_n['age']).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        30|
|  19| Justin|        19|
+----+-------+----------+



In [29]:
df_n.withColumn('double_age',df_n['age']*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [30]:
df_n.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [31]:
df_n.withColumnRenamed('age','my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



##### To use sql query we have to 1st register the dataframe as a sql temporary view

In [32]:
df_n.createOrReplaceTempView('folks')

In [33]:
results  = spark.sql('SELECT * from folks')

In [34]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [35]:
new_results = spark.sql('SELECT * from folks WHERE age=30')

In [36]:
new_results

DataFrame[age: bigint, name: string]

In [37]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

